In [1]:
import os
os.chdir("../")

In [2]:
import glob
import pandas as pd
import time
import numpy as np
from scipy import stats

In [76]:
resultDir = 'results'
problem = 'cauctions' # choices=['setcover', 'cauctions', 'facilities', 'indset']
sampling_Strategies = ['uniform5','depthK','depthK2'] # choices: uniform5, depthK, depthK2, depthK3
seeds = [0,1,2,3,4]

In [77]:
eval_files = glob.glob(f'{resultDir}/{problem}_*.csv')
eval_file = eval_files[-1]

df = pd.read_csv(eval_file)
df = pd.concat([df[df['type']=='small'], df[df['type']=='medium']])
df = df.astype({'nlps': float, 'nnodes' : float})

df_gcnns = df[df['policy'] != 'internal:relpscost'].copy()

In [78]:
def gmean_1shifted(x):
    return stats.mstats.gmean(x + 1) - 1

# 1. Means

In [138]:
dfgcnns_gmean = df_gcnns.groupby(['type','sampling_strategy'])[['nnodes', 'stime']].agg(gmean_1shifted)
dfgcnns_mean = df_gcnns.groupby(['type','sampling_strategy'])[['nnodes', 'stime']].mean()
dfgcnns_gmean

nnodes      stime
type   sampling_strategy                        
medium depthK             1175.979549  59.232734
       depthK2            1189.875678  60.563289
       uniform5           1203.071862  61.204966
small  depthK               67.402727   5.757237
       depthK2              63.955518   5.662016
       uniform5             65.949223   5.805463

In [140]:
def norm_by_uniform5(dfgcnns_grouped):
    df_list = []
    for probSize in dfgcnns_grouped.index.levels[0]:
        df_list.append(dfgcnns_grouped.loc[probSize] / dfgcnns_grouped.loc[(probSize, 'uniform5')])
    return pd.concat(df_list, keys=dfgcnns_grouped.index.levels[0])

In [142]:
dfgcnns_gmean_norm = norm_by_uniform5(dfgcnns_gmean)
dfgcnns_mean_norm = norm_by_uniform5(dfgcnns_mean)
dfgcnns_gmean_norm

nnodes     stime
type   sampling_strategy                    
medium depthK             0.977481  0.967777
       depthK2            0.989031  0.989516
       uniform5           1.000000  1.000000
small  depthK             1.022040  0.991693
       depthK2            0.969769  0.975291
       uniform5           1.000000  1.000000

# 2. Std variances (per instance)

In [147]:
dfgcnns_std_norm_perInstance = df_gcnns.groupby(['type','sampling_strategy','instance']).std() / df_gcnns.groupby(['type','sampling_strategy','instance']).mean()

In [148]:
dfgcnns_std_norm = dfgcnns_std_norm_perInstance.groupby(['type','sampling_strategy'])[['nnodes','stime']].mean()
dfgcnns_std_norm

nnodes     stime
type   sampling_strategy                    
medium depthK             0.119493  0.085076
       depthK2            0.110848  0.088538
       uniform5           0.121829  0.102216
small  depthK             0.164647  0.097555
       depthK2            0.147574  0.087226
       uniform5           0.183908  0.117446

# 做一下 T-Test

# 计算Wins

In [173]:
ttest_res = pd.DataFrame(index=dfgcnns_mean_norm.index, columns=pd.MultiIndex.from_product((['nnodes', 'stime'], ['t_stats', 'p_value'])))
for metric in ['nnodes', 'stime']:
    for probSize in dfgcnns_mean_norm.index.levels[0]: # medium, small
        mean2 = 1
        std2 = dfgcnns_std_norm.at[(probSize,'uniform5'),metric]
        for sampling_strategy in dfgcnns_mean_norm.index.levels[1]: # depthK, depthK2, uniform5
            mean1 = dfgcnns_mean_norm.at[(probSize,sampling_strategy),metric]
            std1_norm = dfgcnns_std_norm.at[(probSize,sampling_strategy),metric]
            std1 = std1_norm * mean1
            t_stats, p_value = stats.ttest_ind_from_stats(mean1, std1, 100, mean2, std2, 100)
            ttest_res.loc[(probSize,sampling_strategy),(metric,)] = [t_stats, p_value]

In [174]:
ttest_res

nnodes               stime           
                          t_stats    p_value  t_stats    p_value
type   sampling_strategy                                        
medium depthK            -2.55442  0.0113884 -2.37075  0.0187122
       depthK2           -1.63907   0.102787 -1.51718   0.130816
       uniform5                 0          1        0          1
small  depthK            -0.52574   0.599657 -1.01789   0.309971
       depthK2           -1.76068  0.0798366 -2.08273  0.0385604
       uniform5                 0          1        0          1

In [84]:
winner_idx = df_gcnns.groupby(['type','instance','seed'])['stime'].agg(pd.Series.idxmin)
winner_idx

type    instance                                                  seed
medium  data/instances/cauctions/transfer_200_1000/instance_1.lp  0       300.0
                                                                  1       301.0
                                                                  2       307.0
                                                                  3       313.0
                                                                  4       304.0
                                                                          ...  
small   data/instances/cauctions/transfer_100_500/instance_9.lp   0       130.0
                                                                  1       126.0
                                                                  2       132.0
                                                                  3       133.0
                                                                  4       124.0
Name: stime, Length: 200, dtype: float64

In [85]:
df_gcnns['wins'] = 0
df_gcnns.loc[winner_idx,'wins'] = 1

In [94]:
dfgcnns_wins = df_gcnns.groupby(['type', 'sampling_strategy'])['wins'].sum()
dfgcnns_wins

type    sampling_strategy
medium  depthK               41
        depthK2              26
        uniform5             33
small   depthK               25
        depthK2              46
        uniform5             29
Name: Wins, dtype: int64

In [ ]:
scipy.stats.ttest_ind_from_stats(0.967777, 0.085076*0.967777, 100, 1, 0.102216, 100)

# 把Means、std 和 Wins合并起来

In [103]:
pd.concat([dfgcnns_gmean_norm, dfgcnns_std_norm, dfgcnns_wins], axis=1, keys=['gmean','std','wins'])

gmean                 std           wins
                            nnodes     stime    nnodes     stime Wins
type   sampling_strategy                                             
medium depthK             0.977481  0.967777  0.119493  0.085076   41
       depthK2            0.989031  0.989516  0.110848  0.088538   26
       uniform5           1.000000  1.000000  0.121829  0.102216   33
small  depthK             1.022040  0.991693  0.164647  0.097555   25
       depthK2            0.969769  0.975291  0.147574  0.087226   46
       uniform5           1.000000  1.000000  0.183908  0.117446   29

In [249]:
output_idx = pd.MultiIndex.from_product((['small', 'medium'], sampling_Strategies), names=['type', 'sampling_strategy'])
output_idx

MultiIndex([( 'small', 'uniform5'),
            ( 'small',   'depthK'),
            ( 'small',  'depthK2'),
            ('medium', 'uniform5'),
            ('medium',   'depthK'),
            ('medium',  'depthK2')],
           names=['type', 'sampling_strategy'])

In [250]:
output_df_mean = dfgcnns_gmean_norm.reindex(output_idx)
output_df_std = dfgcnns_std_norm.reindex(output_idx)
output_df_wins = dfgcnns_wins.reindex(output_idx)
output_df_ttest = ttest_res.reindex(output_idx)

In [251]:
out_df1 = pd.DataFrame(index=output_idx, columns=['nnodes', 'stime', 'wins', 't-stats (p-val)'])
out_df1

nnodes stime wins t-stats (p-val)
type   sampling_strategy                                  
small  uniform5             NaN   NaN  NaN             NaN
       depthK               NaN   NaN  NaN             NaN
       depthK2              NaN   NaN  NaN             NaN
medium uniform5             NaN   NaN  NaN             NaN
       depthK               NaN   NaN  NaN             NaN
       depthK2              NaN   NaN  NaN             NaN

In [252]:
for col_name in ['nnodes', 'stime']:
    out_df1[col_name] = ["%.4fr ± %.2f" % (m, s*100) + "%" for (m, s) in zip(output_df_mean[col_name], output_df_std[col_name])]
out_df1['wins'] = output_df_wins

In [253]:
out_df1['t-stats (p-val)'] = ["%.2f(%.4f)" % (t, p) for (t, p) in zip(output_df_ttest[('stime','t_stats')], output_df_ttest[('stime','p_value')])]

In [254]:
out_df1

nnodes             stime  wins  \
type   sampling_strategy                                             
small  uniform5           1.0000r ± 18.39%  1.0000r ± 11.74%    29   
       depthK             1.0220r ± 16.46%   0.9917r ± 9.76%    25   
       depthK2            0.9698r ± 14.76%   0.9753r ± 8.72%    46   
medium uniform5           1.0000r ± 12.18%  1.0000r ± 10.22%    33   
       depthK             0.9775r ± 11.95%   0.9678r ± 8.51%    41   
       depthK2            0.9890r ± 11.08%   0.9895r ± 8.85%    26   

                         t-stats (p-val)  
type   sampling_strategy                  
small  uniform5             0.00(1.0000)  
       depthK              -1.02(0.3100)  
       depthK2             -2.08(0.0386)  
medium uniform5             0.00(1.0000)  
       depthK              -2.37(0.0187)  
       depthK2             -1.52(0.1308)